# Digit Recognition App

You can use this app in two ways:

1. Select a 32x32 pixel 'single digit' image to recognize.
2. Select a 128x32 pixel 'postal code' image to recognize.

In [1]:
import ipywidgets as w
import pandas as pd

from SimpleCV import *
from ipywidgets import widgets, HBox, VBox
from IPython.display import display
from sklearn.externals import joblib

disp = Display(displaytype='notebook')
app = w.Output()

FILE_NAME = None
IMAGES = None

### Importing Normalizer & Classifier

In [2]:
CLASSIFIER = joblib.load('classifier.joblib')
SCALER = joblib.load('scaler.joblib')
COLUMNS = [u'area', u'contours', u'radius', u'hull_radius', u'centroid_x',
       u'centroid_y', u'weight_0_0', u'weight_0_1', u'weight_0_2',
       u'weight_0_3', u'weight_0_4', u'weight_0_5', u'weight_0_6',
       u'weight_0_7', u'weight_1_0', u'weight_1_1', u'weight_1_2',
       u'weight_1_3', u'weight_1_4', u'weight_1_5', u'weight_1_6',
       u'weight_1_7', u'weight_2_0', u'weight_2_1', u'weight_2_2',
       u'weight_2_3', u'weight_2_4', u'weight_2_5', u'weight_2_6',
       u'weight_2_7', u'weight_3_0', u'weight_3_1', u'weight_3_2',
       u'weight_3_3', u'weight_3_4', u'weight_3_5', u'weight_3_6',
       u'weight_3_7', u'weight_4_0', u'weight_4_1', u'weight_4_2',
       u'weight_4_3', u'weight_4_4', u'weight_4_5', u'weight_4_6',
       u'weight_4_7', u'weight_5_0', u'weight_5_1', u'weight_5_2',
       u'weight_5_3', u'weight_5_4', u'weight_5_5', u'weight_5_6',
       u'weight_5_7', u'weight_6_0', u'weight_6_1', u'weight_6_2',
       u'weight_6_3', u'weight_6_4', u'weight_6_5', u'weight_6_6',
       u'weight_6_7', u'weight_7_0', u'weight_7_1', u'weight_7_2',
       u'weight_7_3', u'weight_7_4', u'weight_7_5', u'weight_7_6',
       u'weight_7_7', u'num_holes']
COLUMNS_TO_NORMALIZE = [u'area', u'contours', u'radius', u'hull_radius', u'centroid_x',
       u'centroid_y', u'weight_0_0', u'weight_0_1', u'weight_0_2',
       u'weight_0_3', u'weight_0_4', u'weight_0_5', u'weight_0_6',
       u'weight_0_7', u'weight_1_0', u'weight_1_1', u'weight_1_2',
       u'weight_1_3', u'weight_1_4', u'weight_1_5', u'weight_1_6',
       u'weight_1_7', u'weight_2_0', u'weight_2_1', u'weight_2_2',
       u'weight_2_3', u'weight_2_4', u'weight_2_5', u'weight_2_6',
       u'weight_2_7', u'weight_3_0', u'weight_3_1', u'weight_3_2',
       u'weight_3_3', u'weight_3_4', u'weight_3_5', u'weight_3_6',
       u'weight_3_7', u'weight_4_0', u'weight_4_1', u'weight_4_2',
       u'weight_4_3', u'weight_4_4', u'weight_4_5', u'weight_4_6',
       u'weight_4_7', u'weight_5_0', u'weight_5_1', u'weight_5_2',
       u'weight_5_3', u'weight_5_4', u'weight_5_5', u'weight_5_6',
       u'weight_5_7', u'weight_6_0', u'weight_6_1', u'weight_6_2',
       u'weight_6_3', u'weight_6_4', u'weight_6_5', u'weight_6_6',
       u'weight_6_7', u'weight_7_0', u'weight_7_1', u'weight_7_2',
       u'weight_7_3', u'weight_7_4', u'weight_7_5', u'weight_7_6',
       u'weight_7_7']

### Image Separation Logic

In [3]:
# Constant single digit image size
SIZE = 32

# Return an array of individual images and their digit from the image object
def get_individual_images(file_name):
    global IMAGES
    img = Image(file_name)
    IMAGES = [
        get_image_obj(crop(img, 0)), 
        get_image_obj(crop(img, 1)), 
        get_image_obj(crop(img, 2)), 
        get_image_obj(crop(img, 3))
    ]

def get_image_obj(img):
    _bin = img.binarize()
    return (None, _bin.dilate().erode(), _bin)

# Crop and return a part of the source image
def crop(img, part):
    return img.crop(SIZE * part, 0, SIZE, SIZE)

### Feature Extraction Logic

In [4]:
def get_weighted_matrix(img):
    COUNT = 8 # GRID SIZE
    T = 5 # THRESHOLD
    CROP = SIZE / COUNT
    m = np.zeros((COUNT, COUNT))
    for y in range(COUNT):
        for x in range(COUNT):
            part = img.crop(x * CROP, y * CROP, CROP, CROP)
            sum = (part.getNumpy()[:,:,0] / 255).sum()
            m[x][y] = sum if sum > T else 0
    return m.T

def extract_features(image):
    # Columns detected using computer vision
    features_dict = {
        "area": [],
        "contours": [],
        "radius": [],
        "circle_dist": [],
        "rect_dist": [],
        "hull_radius": [],
        "aspect_ratio": [],
        "centroid_x": [],
        "centroid_y": [],
        "angle": [],
        "num_holes": []
    }

    column_order = [
        "area",
        "contours",
        "radius",
        "circle_dist",
        "rect_dist",
        "hull_radius",
        "aspect_ratio",
        "centroid_x",
        "centroid_y",
        "angle"
    ]

    # Adding column names for the low-res pixel count matrix
    COUNT = 8
    for x in range(COUNT):
        for y in range(COUNT):
            name = '_'.join(['weight', str(x), str(y)])
            column_order.append(name)
            features_dict[name] = []

    column_order.append("num_holes")
    
    # HACK TO FIND BLOBS
    image[1].show()
    # Find blobs in the preprocessed image
    blobs = image[1].findBlobs()
    if blobs:
        blob = blobs[0]
        features_dict['area'].append(blob.area())
        features_dict['contours'].append(len(blob.contour()))
        features_dict['circle_dist'].append(blob.circleDistance())
        features_dict['rect_dist'].append(blob.rectangleDistance())
        features_dict['radius'].append(blob.radius())
        features_dict['hull_radius'].append(blob.hullRadius())
        features_dict['aspect_ratio'].append(blob.aspectRatio())
        features_dict['centroid_x'].append(blob.centroid()[0])
        features_dict['centroid_y'].append(blob.centroid()[1])
        features_dict['angle'].append(blob.angle())

        # Calculate matrix in the preprocessed image
        m = get_weighted_matrix(image[1])
        COUNT = 8
        for x in range(COUNT):
            for y in range(COUNT):
                features_dict['_'.join(['weight', str(x), str(y)])].append(m[x, y])

        # Find the holes in the 'just binarized' image
        num_holes = len(image[2].embiggen(2).invert().findBlobs()[:-1])
        features_dict['num_holes'].append(num_holes)
    
    return pd.DataFrame(features_dict, columns=column_order)

### Prediction Logic

In [5]:
def predict(index):
    if IMAGES:
        # extract features
        features = extract_features(IMAGES[index])[COLUMNS]
        # normalize
        features[COLUMNS_TO_NORMALIZE] = pd.DataFrame(SCALER.transform(features[COLUMNS_TO_NORMALIZE]), 
                                                      columns=COLUMNS_TO_NORMALIZE)
        # predict
        prediction = CLASSIFIER.predict(features)[0]
        # update UI
        labels_predict[index].value = str(prediction)
        return prediction
    else:
        labels_predict[index].value = str(-1)
        return index

def predict_all(b):
    if IMAGES:
        return [predict(0), predict(1), predict(2), predict(3)]

### Image Selection UI

In [6]:
# Find all png files in a folder
files = glob.glob(os.path.join("../dataset-images/", "*.png"))

def on_image_select(b):
    global FILE_NAME
    FILE_NAME = dropdown_image.value
    image_preview.value = w.Image.from_file(FILE_NAME).value

dropdown_image = w.Dropdown(options=files, description='Image:')
button_select_image = w.Button(description="Use")
button_select_image.on_click(on_image_select)
image_preview = w.Image()

image_selection = VBox([
    HBox([dropdown_image, button_select_image])
])

with app:
    display(image_selection)
    display(image_preview)
on_image_select(None)

### Prediction UI

In [7]:
def show_separation(b):
    if FILE_NAME:
        get_individual_images(FILE_NAME)
        for i in range(4):
            digit_previews[i].clear_output()
            with digit_previews[i]:
                IMAGES[i][1].save(disp)

# Create a button
button_separate = w.Button(description="Separate")
button_separate.on_click(show_separation)

digit_previews = [w.Output(), w.Output(), w.Output(), w.Output()]

buttons_predict = [
    w.Button(description="Predict"),
    w.Button(description="Predict"),
    w.Button(description="Predict"),
    w.Button(description="Predict")
]

labels_predict = [
    w.Label(value='0'),
    w.Label(value='0'),
    w.Label(value='0'),
    w.Label(value='0')
]

buttons_predict[0].on_click(lambda b: predict(0))
buttons_predict[1].on_click(lambda b: predict(1))
buttons_predict[2].on_click(lambda b: predict(2))
buttons_predict[3].on_click(lambda b: predict(3))

button_predict_all = w.Button(description="Predict Postal Code")
button_predict_all.on_click(predict_all)

selection = VBox([
    HBox([button_separate]),
    HBox([
        VBox([buttons_predict[0], digit_previews[0], labels_predict[0]]),
        VBox([buttons_predict[1], digit_previews[1], labels_predict[1]]),
        VBox([buttons_predict[2], digit_previews[2], labels_predict[2]]),
        VBox([buttons_predict[3], digit_previews[3], labels_predict[3]])
    ]),
    button_predict_all
])
    
with app:
    display(selection)

# Digit Recognition App

You can use this app in two ways:

1. Select a 32x32 pixel 'single digit' image to recognize.
2. Select a 128x32 pixel 'postal code' image to recognize.

In [8]:
app

Output()